In [2]:
# shell call for i in {00..07}; do python geos_rt_tt.py 20211211 12 $i; done

import numpy as np

import xarray as xr

import sys

#data_dir='/data/accp/a/snesbitt/geos_tt/'

data_dir=''

datestr=sys.argv[1]

run=sys.argv[2]

fhr=sys.argv[3]

 

print('Calculating GEOS tropopause temperature: '+datestr+' Run: '+run+' Forecast hour: '+fhr)

 

m3d = xr.open_dataset('https://opendap.nccs.nasa.gov/dods/gmao/geos-cf/fcast/met_inst_1hr_g1440x721_p23/met_inst_1hr_g1440x721_p23.'+datestr+'_'+run+'z')

mds = xr.open_dataset('https://opendap.nccs.nasa.gov/dods/gmao/geos-cf/fcast/met_tavg_1hr_g1440x721_x1/met_tavg_1hr_g1440x721_x1.'+datestr+'_'+run+'z')

 

troppb = mds.isel(lev=0,time=int(fhr))['troppb']

 

t = m3d['t'].isel(time=int(fhr))

 

 

 

def interp1d_np(data, x, xi):

    return np.interp(xi, np.flip(x), np.flip(data))

 

 

#interped = interp1d_np(t.isel(lat=0,lon=0), m3d.lev,troppb.isel(lat=0,lon=0))

out = xr.apply_ufunc(

    interp1d_np,  # first the function

    t,

    m3d.lev,

    troppb/100.,  #convert from Pa to hPa to match lev units

    input_core_dims=[["lev"], ["lev"], []],

    exclude_dims=set(("lev",)),

#    output_core_dims=[["lev"]],

    vectorize=True

)

 

out.to_netcdf(data_dir+datestr+'_'+run+'_f'+fhr+'.nc')

IndexError: list index out of range

In [3]:
print(sys.argv)

['/data/keeling/a/yulanh/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/data/keeling/a/yulanh/.local/share/jupyter/runtime/kernel-3789bf00-b766-48ef-97f2-73b78930dfd8.json']
